In [ ]:
from typing import Callable, List
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd
from dataclasses import dataclass

plt.style.use("ggplot")

In [ ]:
def forward_diff(f: Callable, x: float, h: float) -> float:
    return (f(x + h) - f(x)) / h


def backward_diff(f: Callable, x: float, h: float) -> float:
    return (f(x) - f(x - h)) / h


def central_diff(f: Callable, x: float, h: float) -> float:
    return (f(x + h) - f(x - h)) / (2 * h)


def absolute_error(f: Callable, analytical_f: Callable, x: float) -> float:
    return abs(f(x) - analytical_f(x))


def relative_error(f: Callable, analytical_f: Callable, x: float) -> float:
    analytical = analytical_f(x)
    if analytical == 0:
        return 0
    return abs((f(x) - analytical) / analytical)

In [ ]:
@dataclass
class Method:
    name: str
    results: list
    relative_errors: list
    absolute_errors: list
    f: Callable

In [ ]:

def plot_approximation_and_errors(methods: List[Method], x_values: List[float]):
    h = x_values[1] - x_values[0]
    # Two subplots, the axes array is 1-d
    f, axarr = plt.subplots(3, sharex=True, figsize=(8, 12))
    for method in methods:
        axarr[0].plot(x_values, method.results, label=method.name, marker="o")
        axarr[1].plot(x_values, method.absolute_errors, label=method.name, marker="o")
        axarr[2].plot(x_values, method.relative_errors, label=method.name, marker="o")
    axarr[0].set_ylabel("f(x)")
    axarr[1].set_ylabel("Erros Absolutos")
    axarr[2].set_ylabel("Erros Relativos")
    axarr[2].set_xlabel("x")
    axarr[0].set_title(f"Comparação das aproximações e erros para h = {np.round(h, 12)}")
    axarr[0].legend()
    axarr[1].legend()
    axarr[2].legend()
    axarr[0].grid(True)
    axarr[1].grid(True)
    axarr[2].grid(True)
    plt.show()

In [ ]:
def run_approximations(
    f: Callable, f_prime: Callable, a: float, b: float, h_values: float
) -> List[pd.DataFrame]:
    x_values = np.arange(a, b + h, h)
    analytical = Method("Sol. Analítica", [], [], [], f_prime)
    forward = Method("Avançada", [], [], [], lambda x: forward_diff(f, x, h))
    backward = Method("Atrasada", [], [], [], lambda x: backward_diff(f, x, h))
    central = Method("Central", [], [], [], lambda x: central_diff(f, x, h))
    methods = [forward, backward, central, analytical]

    for method in methods:
        for x in x_values:
            method.results.append(method.f(x))
            method.absolute_errors.append(absolute_error(method.f, analytical.f, x))
            method.relative_errors.append(relative_error(method.f, analytical.f, x))

    plot_approximation_and_errors(methods, x_values)

    dataframes = []
    for method in methods:
        df = pd.DataFrame(
            {
                "f(x)": method.results,
                f"{method.name}": method.results,
                "Erro Absoluto": method.absolute_errors,
                "Erro Relativo": method.relative_errors,
            },
            index=x_values,
        )
        dataframes.append(df)

    return dataframes

# Tarefa 3

Dadas as funções abaixo utilize as três fórmulas de diferenças (avançada, atrasada e centrada) para aproximar as suas derivadas, considerando diferentes espaçamentos (valores de $h$). Plote um gráfico comparando a solução exata com as soluções aproximadas e, ainda, calcule o erro local e o erro relativo para cada aproximação.

### Problema A

In [ ]:
f = lambda x: np.sin(np.exp(2 * x))
f_prime = lambda x: 2 * np.exp(2 * x) * np.cos(np.exp(2 * x))
a, b = 0, 1

for h in [1 / 2 ** i for i in range(4, 6)]:
  run_approximations(f, f_prime, a, b, h)

### Problema B

In [ ]:
f = lambda x: np.sin(x) / np.log(x)
f_prime = lambda x: (np.cos(x) * np.log(x) - np.sin(x) / x) / (np.log(x) ** 2)
a, b = 6, 7

for h in [1 / 2 ** i for i in range(2, 5)]:
  run_approximations(f, f_prime, a, b, h)

### Problema C

In [ ]:
f = lambda x: np.log(x) * np.sin(x)
f_prime = lambda x: np.sin(x) / x + np.log(x) * np.cos(x)
a, b = 0.5, 1.5

for h in [1 / 2 ** i for i in range(2, 5)]:
  run_approximations(f, f_prime, a, b, h)

### Problema D

In [ ]:
f = lambda x: np.exp(np.sin(x))
f_prime = lambda x: np.exp(np.sin(x)) * np.cos(x)
a, b = 0.5, 1.5

for h in [1 / 2 ** i for i in range(2, 5)]:
  run_approximations(f, f_prime, a, b, h)

### Problema Proposto

In [ ]:
f = lambda x: np.sin((x**2 + 1) / np.log(x**2 + 1)) * np.exp(x * np.log(x))
f_prime = lambda x: (2*x/np.log(x**2 + 1) - 2*x/np.log(x**2 + 1)**2) * np.exp(x*np.log(x)) * np.cos((x**2 + 1)/np.log(x**2 + 1)) + (np.log(x) + 1) * np.exp(x*np.log(x)) * np.sin((x**2 + 1)/np.log(x**2 + 1))

a, b = 0.5, 2 

for h in [1 / 2 ** i for i in range(3, 6)]:
  run_approximations(f, f_prime, a, b, h)